In [10]:
# ==== 0. INSTALL LIBRARY ====
!pip install tensorflow==2.19.1 mediapipe==0.10.7 protobuf==3.20.3 opencv-python scikit-learn matplotlib seaborn tqdm --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires tensorflow~=2.15.0, but you have tensorflow 2.19.1 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.19.1 which is incompatible.


In [3]:
# ==== 1. IMPORT & SETUP ====
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm
import joblib
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

mp_hands = mp.solutions.hands

In [4]:
# ==== 2. FUNGSI FITUR RELATIF (DENGAN NORMALISASI SKALA) ====
def extract_hand_vector_rel(landmarks):
    """
    Ekstrak 9 fitur berbasis jarak antar jari dan panjang jari,
    dinormalisasi berdasarkan ukuran telapak tangan (palm_size).
    """
    base = landmarks[0]
    rel = landmarks - base
    palm_size = np.linalg.norm(rel[0] - rel[12]) + 1e-6  # biar gak divide by zero

    # Jarak ibu jari ke jari lain (8,12,16,20)
    d_thumb = [np.linalg.norm(rel[4] - rel[i]) / palm_size for i in [8, 12, 16, 20]]

    # Panjang masing-masing jari
    finger_len = [
        np.linalg.norm(rel[0] - rel[4]) / palm_size,
        np.linalg.norm(rel[0] - rel[8]) / palm_size,
        np.linalg.norm(rel[0] - rel[12]) / palm_size,
        np.linalg.norm(rel[0] - rel[16]) / palm_size,
        np.linalg.norm(rel[0] - rel[20]) / palm_size
    ]

    return np.array(d_thumb + finger_len)

In [5]:
# ==== 2. FUNGSI EKSTRAK VEKTOR RELATIF ====
def extract_hand_vector_rel(landmarks):
    """Hitung fitur jarak antar jari dan panjang jari"""
    base = landmarks[0]
    rel = landmarks - base
    
    # Jarak antara ibu jari ke jari lain (4 → 8,12,16,20)
    d_thumb = [np.linalg.norm(rel[4] - rel[i]) for i in [8,12,16,20]]

    # Panjang masing-masing jari (pergelangan ke ujung jari)
    finger_len = [
        np.linalg.norm(rel[0] - rel[4]),   # ibu jari
        np.linalg.norm(rel[0] - rel[8]),   # telunjuk
        np.linalg.norm(rel[0] - rel[12]),  # tengah
        np.linalg.norm(rel[0] - rel[16]),  # manis
        np.linalg.norm(rel[0] - rel[20])   # kelingking
    ]

    vec = np.array(d_thumb + finger_len)
    return vec

In [ ]:
# ==== 3. EKSTRAKSI DATASET ====
dataset_dir = "./SIBI/number"  
output_dir = "./SIBI/number_vector"
os.makedirs(output_dir, exist_ok=True)

labels_list = sorted(os.listdir(dataset_dir))
label_map = {label: i for i, label in enumerate(labels_list)}

X, y = [], []

with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.6) as hands:
    for label in tqdm(labels_list, desc="Ekstraksi dataset"):
        folder = os.path.join(dataset_dir, label)
        for file in os.listdir(folder):
            img_path = os.path.join(folder, file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            result = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            if not result.multi_hand_landmarks:
                continue
            hand = result.multi_hand_landmarks[0]
            pts = np.array([[lm.x, lm.y, lm.z] for lm in hand.landmark])
            vec = extract_hand_vector_rel(pts)

            X.append(vec)
            y.append(label_map[label])

X = np.array(X)
y = np.array(y)


Ekstraksi dataset: 100%|██████████| 24/24 [04:10<00:00, 10.43s/it]


In [2]:
# ==== 4. SIMPAN DATASET VECTOR ====
np.save(os.path.join(output_dir, "X_vectors.npy"), X)
np.save(os.path.join(output_dir, "y_labels.npy"), y)
np.save(os.path.join(output_dir, "label_map.npy"), label_map)

print(f"Dataset vektor tersimpan di: {output_dir}")
print(f"   Jumlah sampel: {len(X)} | Fitur: {X.shape[1]} | Kelas: {len(labels_list)}")

NameError: name 'os' is not defined

In [6]:
# ==== 5. LOAD DATASET ====
X = np.load("./SIBI/number_vector/X_vectors.npy")
y = np.load("./SIBI/number_vector/y_labels.npy")
label_map = np.load("./SIBI/number_vector/label_map.npy", allow_pickle=True).item()
labels_list = list(label_map.keys())

print("Data loaded:", X.shape, " | Classes:", labels_list)

Data loaded: (1650, 9)  | Classes: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [7]:
# ==== 6. SPLIT TRAIN-TEST ====
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [8]:
# ==== 7. TRAIN MODEL ====
model = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=800, random_state=42)
model.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=800, random_state=42)

In [9]:
# ==== 8. EVALUASI ====
y_pred = model.predict(X_test)
print("\n Akurasi:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=labels_list))



 Akurasi: 0.9939393939393939
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        29
           1       1.00      0.97      0.98        30
           2       0.96      1.00      0.98        24
           3       1.00      1.00      1.00        39
           4       1.00      1.00      1.00        34
           5       1.00      1.00      1.00        42
           6       1.00      1.00      1.00        29
           7       1.00      1.00      1.00        29
           8       1.00      1.00      1.00        34
           9       1.00      1.00      1.00        40

    accuracy                           0.99       330
   macro avg       0.99      0.99      0.99       330
weighted avg       0.99      0.99      0.99       330



In [ ]:
# ==== 9. SIMPAN MODEL ====
os.makedirs("model_vector", exist_ok=True)
joblib.dump(model, "model_vector/hand_vector_model.pkl")
np.save("model_vector/label_map.npy", label_map)
print(" Model disimpan di model_vector/")

💾 Model disimpan di model_vector/


In [10]:
# ==== REALTIME HAND SIGN CLASSIFICATION (1 FPS, FITUR RELATIF) ====
import cv2
import numpy as np
import mediapipe as mp
import joblib
import time

# ==== 1. LOAD MODEL ====
model = joblib.load("model_vector/mauna_number_model.pkl")
label_map = np.load("model_vector/mauna_number_label_map.npy", allow_pickle=True).item()
inv_label = {v: k for k, v in label_map.items()}

# ==== 2. SETUP MEDIAPIPE ====
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# ==== 3. FUNGSI FITUR RELATIF (harus sama kayak di training) ====
def extract_hand_vector_rel(landmarks):
    base = landmarks[0]
    rel = landmarks - base
    d_thumb = [np.linalg.norm(rel[4] - rel[i]) for i in [8,12,16,20]]
    finger_len = [
        np.linalg.norm(rel[0] - rel[4]),
        np.linalg.norm(rel[0] - rel[8]),
        np.linalg.norm(rel[0] - rel[12]),
        np.linalg.norm(rel[0] - rel[16]),
        np.linalg.norm(rel[0] - rel[20])
    ]
    return np.array(d_thumb + finger_len)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError(" Kamera tidak terdeteksi!")

print("[INFO] Jalankan klasifikasi 1x per detik (ESC untuk keluar)...")

last_time = 0
pred_label, pred_conf = "...", 0.0

with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)

        now = time.time()
        if now - last_time >= 1.0:  # hanya prediksi setiap 1 detik
            last_time = now
            result = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if result.multi_hand_landmarks:
                hand = result.multi_hand_landmarks[0]
                pts = np.array([[lm.x, lm.y, lm.z] for lm in hand.landmark])
                vec = extract_hand_vector_rel(pts)
                probs = model.predict_proba([vec])[0]
                idx = np.argmax(probs)
                pred_label = inv_label[idx]
                pred_conf = probs[idx]
                print(f"[{time.strftime('%H:%M:%S')}] {pred_label} ({pred_conf*100:.1f}%)")

        # Gambarkan tangan (optional tiap frame)
        result = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if result.multi_hand_landmarks:
            for hand in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand, mp_hands.HAND_CONNECTIONS)

        cv2.putText(frame, f"{pred_label} ({pred_conf*100:.1f}%)", (30,80),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 3)
        cv2.imshow("SIBI Vector (1 FPS, Relatif)", frame)

        if cv2.waitKey(1) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


[INFO] Jalankan klasifikasi 1x per detik (ESC untuk keluar)...
[12:50:14] 5 (99.9%)
[12:50:15] 1 (66.0%)
[12:50:16] 1 (99.1%)
[12:50:17] 2 (99.9%)
[12:50:18] 2 (99.9%)
[12:50:19] 3 (70.3%)
[12:50:20] 3 (57.8%)
[12:50:21] 4 (99.9%)
[12:50:22] 4 (99.9%)
[12:50:23] 5 (100.0%)
[12:50:24] 5 (100.0%)
[12:50:25] 6 (99.9%)
[12:50:26] 6 (99.9%)
[12:50:27] 7 (100.0%)
[12:50:28] 7 (100.0%)
[12:50:29] 8 (100.0%)
[12:50:30] 8 (100.0%)
[12:50:31] 9 (100.0%)
[12:50:32] 0 (99.7%)
[12:50:33] 0 (99.9%)
[12:50:34] 4 (34.9%)
[12:50:37] 5 (99.9%)
[12:50:38] 1 (99.8%)
[12:50:39] 1 (99.6%)
[12:50:40] 2 (100.0%)
[12:50:41] 3 (75.7%)
[12:50:42] 3 (89.7%)
[12:50:43] 4 (100.0%)
[12:50:44] 4 (100.0%)
[12:50:45] 5 (100.0%)
[12:50:46] 5 (100.0%)
[12:50:47] 6 (100.0%)
[12:50:48] 7 (98.7%)
[12:50:49] 7 (100.0%)
[12:50:51] 8 (100.0%)
[12:50:52] 5 (61.0%)
[12:50:53] 9 (100.0%)
[12:50:54] 9 (100.0%)
[12:50:56] 0 (99.9%)
[12:50:57] 0 (99.8%)
